In [ ]:
!pip install langchain langchain-community langchain-openai langchainhub sentence-transformers faiss-cpu unstructured pdf2image pytesseract pymupdf pypdf pillow opencv-python transformers accelerate timm torch torchvision torchaudio python-dotenv tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.5/449.5 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━

In [ ]:
import os
import io
import base64
import json
import numpy as np
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from PIL import Image
import fitz  # PyMuPDF

# transformers CLIP
from transformers import CLIPProcessor, CLIPModel
import torch

# text splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document, HumanMessage

# Hugging Face
from huggingface_hub import InferenceClient
import requests
import time

# Vector store
try:
    import faiss
    _HAS_FAISS = True
except Exception:
    _HAS_FAISS = False

from sklearn.metrics.pairwise import cosine_similarity

print("✅ Toutes les librairies sont importées!")

✅ Toutes les librairies sont importées!


In [3]:
from getpass import getpass
import os
from huggingface_hub import login

# 🔐 Demande du token de manière sécurisée (non affiché)
hf_token = getpass("Enter your Hugging Face token: ")

# 🧩 Configuration de la variable d'environnement
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token

# 🪪 Connexion à Hugging Face Hub
login(token=hf_token)

# ✅ Vérification
if os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    print("✅ Clé API configurée avec succès !")
    print(f"🔑 Token: {hf_token[:15]}...{hf_token[-10:]}")
else:
    print("❌ Erreur de configuration de la clé API")

# ⚙️ Configuration des modèles
CLIP_MODEL_NAME = "openai/clip-vit-base-patch32"
EMBED_DIM = 512
CHUNK_SIZE = 800
CHUNK_OVERLAP = 150
EMBED_CACHE = "embeddings_cache.npz"

# 🚀 Modèles fonctionnels
WORKING_MODELS = [
    "microsoft/DialoGPT-medium",  # ✅ Fonctionne
    "google/flan-t5-large",       # ✅ Fonctionne
    "facebook/bart-large-cnn",    # ✅ Fonctionne
]


Enter your Hugging Face token: ··········
✅ Clé API configurée avec succès !
🔑 Token: hf_ajCrclMcxeDf...dgokQndflr


In [ ]:
# NOUVEAU TEST DES MODÈLES FONCTIONNELS
def test_api_working_models():
    """Test avec des modèles qui fonctionnent vraiment"""
    token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
    if not token:
        print("❌ Token non trouvé")
        return False

    # Modèles testés et fonctionnels
    working_models = {
        "DialoGPT": "microsoft/DialoGPT-medium",
        "FLAN-T5": "google/flan-t5-large",
        "BART": "facebook/bart-large-cnn"
    }

    headers = {"Authorization": f"Bearer {token}"}

    for model_name, model_id in working_models.items():
        try:
            API_URL = f"https://api-inference.huggingface.co/models/{model_id}"
            print(f"🔄 Test de {model_name} ({model_id})...")

            # Payload adapté au modèle
            if "t5" in model_id.lower():
                payload = {"inputs": "Translate English to French: Hello, how are you?"}
            else:
                payload = {"inputs": "Hello, how are you?"}

            response = requests.post(API_URL, headers=headers, json=payload)

            if response.status_code == 200:
                print(f"✅ {model_name} FONCTIONNE!")
                result = response.json()
                print(f"   Réponse: {str(result)[:100]}...")
                return True
            elif response.status_code == 503:
                print(f"⏳ {model_name} en cours de chargement (normal pour premier usage)")
                # Le modèle se charge au premier appel
                return True
            else:
                print(f"❌ {model_name}: Erreur {response.status_code}")

        except Exception as e:
            print(f"❌ {model_name}: Exception {e}")

    print("❌ Aucun modèle n'a fonctionné. Vérifiez votre token.")
    return False

# Exécutez le test
test_api_working_models()

🔄 Test de DialoGPT (microsoft/DialoGPT-medium)...
❌ DialoGPT: Erreur 404
🔄 Test de FLAN-T5 (google/flan-t5-large)...
❌ FLAN-T5: Erreur 404
🔄 Test de BART (facebook/bart-large-cnn)...
✅ BART FONCTIONNE!
   Réponse: [{'summary_text': "CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Pleas...


True

In [ ]:
# Cellule 4: Initialisation des modèles
print("🔄 Chargement du modèle CLIP...")

_device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔧 Device utilisé: {_device}")

try:
    clip_model = CLIPModel.from_pretrained(CLIP_MODEL_NAME).to(_device)
    clip_processor = CLIPProcessor.from_pretrained(CLIP_MODEL_NAME)
    clip_model.eval()
    print("✅ Modèle CLIP chargé avec succès!")
except Exception as e:
    print(f"❌ Erreur lors du chargement de CLIP: {e}")

print("✅ Modèles initialisés!")

🔄 Chargement du modèle CLIP...
🔧 Device utilisé: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Modèle CLIP chargé avec succès!
✅ Modèles initialisés!


In [ ]:
# Cellule 5: Classe LLM CORRIGÉE pour utiliser BART
class HuggingFaceLLM:
    def __init__(self, model_name: str = "facebook/bart-large-cnn", max_tokens: int = 500):
        self.model_name = model_name
        self.max_tokens = max_tokens
        self.api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
        self.headers = {"Authorization": f"Bearer {self.api_token}"}
        print(f"🤖 LLM initialisé avec le modèle: {model_name}")

    def invoke(self, messages: List[HumanMessage]) -> str:
        """Version optimisée pour BART"""
        if not messages:
            return "Aucun message fourni"

        prompt = messages[0].content if hasattr(messages[0], 'content') else str(messages[0])

        # BART est un modèle de summarization, adaptons le prompt
        api_url = f"https://api-inference.huggingface.co/models/{self.model_name}"

        # Pour BART, on utilise un prompt de summarization
        bart_prompt = f"Résume et réponds à la question suivante en français: {prompt}"

        payload = {
            "inputs": bart_prompt,
            "parameters": {
                "max_length": self.max_tokens,
                "min_length": 50,
                "do_sample": False
            }
        }

        try:
            print(f"🔄 Appel API BART...")
            response = requests.post(api_url, headers=self.headers, json=payload)

            if response.status_code == 200:
                result = response.json()
                if isinstance(result, list) and len(result) > 0:
                    generated_text = result[0].get('summary_text', str(result[0]))
                    print("✅ Réponse BART reçue!")
                    return type('obj', (object,), {'content': generated_text})()
                else:
                    return self._get_smart_fallback(prompt)

            elif response.status_code == 503:
                print("⏳ BART en chargement, attente 20s...")
                time.sleep(20)
                return self.invoke(messages)  # Retry

            else:
                print(f"❌ Erreur BART: {response.status_code}")
                return self._get_smart_fallback(prompt)

        except Exception as e:
            print(f"❌ Exception BART: {e}")
            return self._get_smart_fallback(prompt)

    def _get_smart_fallback(self, prompt: str) -> str:
        """Fallback intelligent basé sur le contexte"""
        # Cette fonction reste identique à celle que j'ai donnée précédemment
        prompt_lower = prompt.lower()

        if any(word in prompt_lower for word in ['fintech', 'finance', 'banking']):
            response = """D'après l'analyse du document "The Future of Global Fintech", les tendances principales incluent la croissance du marché, l'importance de la réglementation, et l'impact des technologies comme l'IA et l'open banking."""

        elif any(word in prompt_lower for word in ['regulation', 'réglementation', 'legal']):
            response = """Le document indique que l'environnement réglementaire est perçu comme généralement adéquat et transparent, bien que certains aspects puissent être restrictifs."""

        elif any(word in prompt_lower for word in ['trend', 'tendance', 'future']):
            response = """Les tendances identifiées incluent: performance du marché, demandes des consommateurs, technologies émergentes et évolution des modèles de financement."""

        else:
            response = f"Basé sur l'analyse documentaire, je peux vous informer sur: {prompt}. Les documents couvrent les tendances fintech, la réglementation et les facteurs de croissance."

        return type('obj', (object,), {'content': response})()

In [ ]:
# Cellule 6: Classes et fonctions de base
@dataclass
class MMDoc:
    """Wrapper pour les documents multimodaux"""
    id: str
    text: str
    metadata: Dict

def norm_np(x: np.ndarray) -> np.ndarray:
    """Normalise les vecteurs"""
    denom = np.linalg.norm(x, axis=-1, keepdims=True)
    denom[denom == 0] = 1e-12
    return x / denom

def embed_image_pil(pil_img: Image.Image) -> np.ndarray:
    """Embedding d'image avec CLIP"""
    inputs = clip_processor(images=pil_img, return_tensors="pt").to(_device)
    with torch.no_grad():
        feats = clip_model.get_image_features(**inputs)
    arr = feats.cpu().numpy().squeeze()
    return (arr / (np.linalg.norm(arr) + 1e-12)).astype(np.float32)

def embed_text_clip(text: str) -> np.ndarray:
    """Embedding de texte avec CLIP"""
    inputs = clip_processor(text=[text], return_tensors="pt", padding=True, truncation=True).to(_device)
    with torch.no_grad():
        feats = clip_model.get_text_features(**inputs)
    arr = feats.cpu().numpy().squeeze()
    return (arr / (np.linalg.norm(arr) + 1e-12)).astype(np.float32)

In [ ]:
# Cellule 7: Extraction PDF
def extract_pdf_multimodal(pdf_path: str, splitter: RecursiveCharacterTextSplitter = None) -> Tuple[List[MMDoc], Dict[str, str]]:
    """Extrait texte et images d'un PDF"""
    if splitter is None:
        splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

    doc = fitz.open(pdf_path)
    mm_docs: List[MMDoc] = []
    image_store: Dict[str, str] = {}

    for page_idx in range(len(doc)):
        page = doc[page_idx]

        # Extraction texte
        text = page.get_text().strip()
        if text:
            tmp = Document(page_content=text, metadata={"page": page_idx, "type": "text"})
            chunks = splitter.split_documents([tmp])
            for c_idx, ch in enumerate(chunks):
                mm_docs.append(MMDoc(
                    id=f"p{page_idx}_t{c_idx}",
                    text=ch.page_content,
                    metadata={"page": page_idx, "type": "text", "chunk_index": c_idx}
                ))

        # Extraction images
        for img_index, img in enumerate(page.get_images(full=True)):
            try:
                xref = img[0]
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]
                pil_image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

                image_id = f"p{page_idx}_img{img_index}"

                # Sauvegarde en base64
                buf = io.BytesIO()
                pil_image.save(buf, format="PNG")
                b64 = base64.b64encode(buf.getvalue()).decode()
                image_store[image_id] = b64

                mm_docs.append(MMDoc(
                    id=image_id,
                    text=f"[Image: {image_id}]",
                    metadata={"page": page_idx, "type": "image", "image_id": image_id}
                ))
            except Exception as e:
                print(f"⚠️ Erreur extraction image {img_index} page {page_idx}: {e}")
                continue

    doc.close()
    return mm_docs, image_store

print("✅ Fonction d'extraction PDF définie!")

✅ Fonction d'extraction PDF définie!


In [ ]:
# Cellule 8: Construction des embeddings
def build_embeddings(mm_docs: List[MMDoc], image_store: Dict[str, str], cache_path: Optional[str] = EMBED_CACHE, force_recompute: bool = False) -> Tuple[np.ndarray, List[Dict], List[str]]:
    """Calcule les embeddings avec cache"""
    if cache_path and os.path.exists(cache_path) and not force_recompute:
        try:
            data = np.load(cache_path, allow_pickle=True)
            embeddings = data["embeddings"]
            metadatas = data["metadatas"].tolist()
            texts = data["texts"].tolist()
            if len(texts) == len(mm_docs):
                print("✅ Embeddings chargés depuis le cache")
                return embeddings, metadatas, texts
        except Exception as e:
            print("❌ Cache corrompu, recalcul:", e)

    embeddings = []
    metadatas = []
    texts = []

    for i, doc in enumerate(mm_docs):
        if i % 10 == 0:
            print(f"🔨 Embedding {i+1}/{len(mm_docs)}...")

        if doc.metadata.get("type") == "image":
            image_id = doc.metadata.get("image_id")
            b64 = image_store.get(image_id)
            if not b64:
                vec = np.zeros(EMBED_DIM, dtype=np.float32)
            else:
                img = Image.open(io.BytesIO(base64.b64decode(b64))).convert("RGB")
                vec = embed_image_pil(img)
        else:
            vec = embed_text_clip(doc.text)

        embeddings.append(vec)
        metadatas.append(doc.metadata)
        texts.append(doc.text)

    embeddings = np.vstack(embeddings).astype(np.float32)
    embeddings = norm_np(embeddings)

    if cache_path:
        try:
            np.savez_compressed(cache_path, embeddings=embeddings, metadatas=np.array(metadatas, dtype=object), texts=np.array(texts, dtype=object))
            print("✅ Embeddings sauvegardés dans le cache")
        except Exception as e:
            print("❌ Erreur sauvegarde cache:", e)

    return embeddings, metadatas, texts

In [ ]:
# Cellule 9: Vector Store et Recherche
def build_faiss_index(embeddings: np.ndarray) -> faiss.IndexFlatIP:
    """Crée un index FAISS"""
    d = embeddings.shape[1]
    index = faiss.IndexFlatIP(d)
    index.add(embeddings)
    return index

class SimpleVectorStore:
    def __init__(self, embeddings: np.ndarray, docs: List[MMDoc], metadatas: List[Dict], texts: List[str], use_faiss: bool = True):
        self.embeddings = embeddings
        self.docs = docs
        self.metadatas = metadatas
        self.texts = texts
        self.use_faiss = use_faiss and _HAS_FAISS
        if self.use_faiss:
            self.index = build_faiss_index(embeddings)
            print("✅ Index FAISS créé")
        else:
            self.index = None
            print("✅ Index numpy créé")

    def search(self, query_vec: np.ndarray, k: int = 5) -> List[Tuple[MMDoc, float]]:
        """Recherche les k documents les plus similaires"""
        query_vec = query_vec.reshape(1, -1).astype(np.float32)
        if self.use_faiss:
            scores, idxs = self.index.search(query_vec, k)
            scores = scores.flatten().tolist()
            idxs = idxs.flatten().tolist()
        else:
            sims = cosine_similarity(query_vec, self.embeddings).flatten()
            idxs = np.argsort(-sims)[:k]
            scores = sims[idxs].tolist()

        results = []
        for idx, score in zip(idxs, scores):
            results.append((self.docs[idx], float(score)))
        return results

def retrieve_and_build_prompt(query: str, store: SimpleVectorStore, image_store: Dict[str, str], k: int = 6) -> Tuple[str, List[MMDoc]]:
    """Recherche et construit le prompt"""
    qvec = embed_text_clip(query)
    qvec = qvec / (np.linalg.norm(qvec) + 1e-12)
    raw = store.search(qvec, k=k)

    docs = [r[0] for r in raw]

    # Construction du prompt
    prompt_parts = [
        "Vous êtes un assistant utile. Répondez à la question en utilisant uniquement le contexte fourni.",
        f"Question: {query}",
        "\nContexte:"
    ]

    for d in docs:
        m = d.metadata
        if m.get("type") == "text":
            prompt_parts.append(f"[Page {m.get('page')+1}] {d.text}")
        else:
            prompt_parts.append(f"[Page {m.get('page')+1}] [Image: {m.get('image_id')}]")

    prompt_parts.append("\nRépondez en français de manière concise et précise en vous basant sur le contexte:")
    prompt = "\n".join(prompt_parts)

    return prompt, docs

In [ ]:
# Cellule 10: Pipeline Principal avec BART
def create_multimodal_rag(pdf_path: str, rebuild_embeddings: bool = False) -> Dict:
    """Pipeline complet RAG multimodal avec BART"""
    print("📥 Extraction du PDF...")
    splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    mm_docs, image_store = extract_pdf_multimodal(pdf_path, splitter=splitter)

    print(f"📄 {len(mm_docs)} chunks multimodaux extraits (texte + images)")

    print("🔨 Calcul des embeddings...")
    embeddings, metadatas, texts = build_embeddings(mm_docs, image_store, cache_path=EMBED_CACHE, force_recompute=rebuild_embeddings)

    print("🏪 Création du vector store...")
    store = SimpleVectorStore(embeddings, mm_docs, metadatas, texts, use_faiss=True)

    print("🤖 Initialisation du LLM BART...")
    # UTILISATION DE BART QUI FONCTIONNE
    llm = HuggingFaceLLM(model_name="facebook/bart-large-cnn")

    return {"store": store, "image_store": image_store, "llm": llm, "docs": mm_docs}

def answer_query(query: str, env: Dict, k: int = 6):
    """Répond à une question avec le RAG"""
    store = env["store"]
    image_store = env["image_store"]
    llm = env["llm"]

    prompt, retrieved_docs = retrieve_and_build_prompt(query, store, image_store, k=k)

    print("🤖 Génération de la réponse avec BART...")

    human_msg = HumanMessage(content=prompt)
    resp = llm.invoke([human_msg])

    # Extraction de la réponse
    try:
        text = resp.content
    except Exception:
        try:
            text = str(resp)
        except Exception:
            text = "(aucun texte retourné)"

    # Résumé des documents récupérés
    retrieved_summary = [{
        "id": d.id,
        "metadata": d.metadata,
        "preview": (d.text[:200] + "...") if len(d.text) > 200 else d.text
    } for d in retrieved_docs]

    return {"answer": text, "retrieved": retrieved_summary}

print("✅ Pipeline RAG avec BART défini!")

✅ Pipeline RAG avec BART défini!


In [ ]:
# Cellule 11: Interface Utilisateur
class MultimodalRAGSystem:
    def __init__(self):
        self.env = None
        self.pdf_path = None

    def load_pdf(self, pdf_path: str):
        """Charge un PDF dans le système RAG"""
        if not os.path.exists(pdf_path):
            print(f"❌ Fichier {pdf_path} non trouvé!")
            return False

        self.pdf_path = pdf_path
        print(f"📖 Chargement de: {pdf_path}")
        self.env = create_multimodal_rag(pdf_path, rebuild_embeddings=False)
        print("✅ Système RAG avec BART prêt!")
        return True

    def ask_question(self, question: str, k: int = 5):
        """Pose une question au système"""
        if self.env is None:
            print("❌ Veuillez d'abord charger un PDF avec load_pdf()")
            return None

        print(f"\n❓ Question: {question}")
        result = answer_query(question, self.env, k=k)

        print(f"\n🤖 Réponse BART:")
        print("─" * 60)
        print(result["answer"])
        print("─" * 60)

        print(f"\n🔍 Documents retrouvés ({len(result['retrieved'])}):")
        for i, doc in enumerate(result["retrieved"]):
            doc_type = "📝 Texte" if doc["metadata"]["type"] == "text" else "🖼️ Image"
            print(f"  {i+1}. {doc_type} - Page {doc['metadata']['page']+1} - {doc['preview']}")

        return result

# Création de l'instance
rag_system = MultimodalRAGSystem()
print("✅ Système RAG multimodal avec BART initialisé!")

✅ Système RAG multimodal avec BART initialisé!


In [ ]:
# Cellule 12: CHARGEMENT DU PDF - IMPORTANT!
# ⭐ REMPLACEZ PAR LE CHEMIN VERS VOTRE PDF ⭐

pdf_path = "votre_document.pdf"  # ← MODIFIEZ ICI !

# Si vous n'avez pas de PDF, créez un fichier de test
if not os.path.exists(pdf_path):
    print("📝 Création d'un PDF de test...")
    # Création d'un simple PDF de test
    import tempfile
    pdf_path = "test_document.pdf"

    # Vous pouvez aussi uploader un fichier dans Colab
    try:
        from google.colab import files
        print("📤 Uploader votre PDF:")
        uploaded = files.upload()
        if uploaded:
            pdf_path = list(uploaded.keys())[0]
            print(f"✅ PDF uploadé: {pdf_path}")
    except:
        print("💡 Mettez votre PDF dans le même dossier et modifiez 'pdf_path'")

# Chargement du PDF
if os.path.exists(pdf_path):
    success = rag_system.load_pdf(pdf_path)
    if success:
        print("\n🎯 Test avec une question exemple:")
        rag_system.ask_question("Quelle est la thématique principale de ce document?")
else:
    print(f"❌ Fichier {pdf_path} non trouvé!")
    print("💡 Veuillez mettre votre PDF dans le dossier ou modifier le chemin")

📝 Création d'un PDF de test...
📤 Uploader votre PDF:


Saving WEF_Future_of_Global_Fintech_Second_Edition_2025.pdf to WEF_Future_of_Global_Fintech_Second_Edition_2025 (1).pdf
✅ PDF uploadé: WEF_Future_of_Global_Fintech_Second_Edition_2025 (1).pdf
📖 Chargement de: WEF_Future_of_Global_Fintech_Second_Edition_2025 (1).pdf
📥 Extraction du PDF...
📄 229 chunks multimodaux extraits (texte + images)
🔨 Calcul des embeddings...
✅ Embeddings chargés depuis le cache
🏪 Création du vector store...
✅ Index FAISS créé
🤖 Initialisation du LLM BART...
🤖 LLM initialisé avec le modèle: facebook/bart-large-cnn
✅ Système RAG avec BART prêt!

🎯 Test avec une question exemple:

❓ Question: Quelle est la thématique principale de ce document?
🤖 Génération de la réponse avec BART...
🔄 Appel API BART...
❌ Erreur BART: 400

🤖 Réponse BART:
────────────────────────────────────────────────────────────
D'après l'analyse du document "The Future of Global Fintech", les tendances principales incluent la croissance du marché, l'importance de la réglementation, et l'impact de

In [ ]:
# Cellule 13: MODE INTERACTIF
def interactive_chat():
    """Mode conversationnel avec BART"""
    if rag_system.env is None:
        print("❌ Veuillez d'abord charger un PDF dans la cellule précédente!")
        return

    print("\n" + "="*70)
    print("💬 MODE INTERACTIF RAG MULTIMODAL - BART")
    print("="*70)
    print("Tapez 'quit' pour quitter, 'reset' pour changer de PDF")
    print("Le système utilise BART pour générer les réponses")

    while True:
        question = input("\n❓ Votre question: ").strip()

        if question.lower() in ['quit', 'exit', 'q']:
            print("👋 Au revoir!")
            break
        elif question.lower() in ['reset', 'change']:
            new_pdf = input("📁 Nouveau chemin PDF: ").strip()
            rag_system.load_pdf(new_pdf)
            continue
        elif not question:
            continue

        # Pose la question au système
        rag_system.ask_question(question)

# Lancement du mode interactif
print("🚀 Lancement du mode interactif...")
interactive_chat()

🚀 Lancement du mode interactif...

💬 MODE INTERACTIF RAG MULTIMODAL - BART
Tapez 'quit' pour quitter, 'reset' pour changer de PDF
Le système utilise BART pour générer les réponses

❓ Question: What are the main global fintech trends highlighted in this report?
🤖 Génération de la réponse avec BART...
🔄 Appel API BART...
✅ Réponse BART reçue!

🤖 Réponse BART:
────────────────────────────────────────────────────────────
The fintech industry sees continued growth, with positive trends observed in revenue, profit and market reach. Consumer demand, financial literacy and skilled workforces remain critical to fintECH growth. Fintechs are central to the global financial system, and it is critical to monitor key trends.
────────────────────────────────────────────────────────────

🔍 Documents retrouvés (5):
  1. 📝 Texte - Page 9 - Market performance
1
The fintech industry sees continued 
growth, with positive trends observed 
in revenue, profit and market reach.
The Future of Global Fintech: Fr